# PySpark review - SQL Dataframe

## PySpark setup

Initially installed with conda but was getting a `Py4JJavaError` that seemed to be related with the version of `pyarrow`, which wasn't fixed after trying multiple versions of the latter, all supported according to the `PySpark` documentation. The error didn't show after I installed (with PIP) PySpark 3.3.0 in base Python for Linux and the following dependencies: Pandas, Pyarrow. 

## Review

### Spark Session

Based on [PySpark documentation](https://spark.apache.org/docs/latest/api/python/getting_started/index.html) and [PySpark API reference](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html).

In [3]:
from pyspark.sql import SparkSession, Row
from datetime import datetime, date
import pandas as pd
# os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

In [4]:
spark = SparkSession.builder.getOrCreate()

22/06/20 08:47:01 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.17.183.216 instead (on interface eth0)
22/06/20 08:47:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/20 08:47:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read data

Read from file

In [108]:
df = spark.read.csv('deniro.csv', header=True, schema='Year integer, Score integer, Title string')
df.show(5)

+----+-----+------------+
|Year|Score|       Title|
+----+-----+------------+
|1968|   86|   Greetings|
|1970|   17| Bloody Mama|
|1970|   73|    Hi, Mom!|
|1971|   40| Born to Win|
|1973|   98|Mean Streets|
+----+-----+------------+
only showing top 5 rows



Create PySpark DataFrame from list of rows

In [123]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

Create PySpark DataFrame with explicit schema

In [6]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

Create PySpark DataFrame from Pandas DataFrame

In [8]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

Create PySpark DataFrame from RDD

In [9]:
rdd = spark.sparkContext.parallelize([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
])
df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

### View data

Print schema

In [6]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



Show data

In [ ]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



Show in Pandas

In [24]:
df.toPandas()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


Show first 2 rows

In [21]:
df.show(2)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 2 rows



Show data vertically

In [23]:
df.show(vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
-RECORD 1------------------
 a   | 2                   
 b   | 3.0                 
 c   | string2             
 d   | 2000-02-01          
 e   | 2000-01-02 12:00:00 
-RECORD 2------------------
 a   | 3                   
 b   | 4.0                 
 c   | string3             
 d   | 2000-03-01          
 e   | 2000-01-03 12:00:00 



Select columns

In [22]:
df.select('a', 'b', 'c').show()

+---+---+-------+
|  a|  b|      c|
+---+---+-------+
|  1|2.0|string1|
|  2|3.0|string2|
|  3|4.0|string3|
+---+---+-------+



### Data wrangling

Describe DataFrame

In [50]:
df.describe().show()

+-------+------------------+------------------+-------------------+
|summary|              Year|             Score|              Title|
+-------+------------------+------------------+-------------------+
|  count|                87|                87|                 87|
|   mean|1995.9770114942528| 57.04597701149425|             1900.0|
| stddev| 12.93901348769487|28.381499069663775|               null|
|    min|              1968|                00|         15 Minutes|
|    max|              2016|                99|What Just Happened?|
+-------+------------------+------------------+-------------------+



Add new column that is c uppercased

In [58]:
from pyspark.sql.functions import upper
df.withColumn('upper_c', upper(df.c)).show()

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e|upper_c|
+---+---+-------+----------+-------------------+-------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|STRING1|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|STRING2|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|STRING3|
+---+---+-------+----------+-------------------+-------+



Select rows where column `a` equals 1

In [9]:
df.filter(df.a == 1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



Add 1 to each row in `a` using Pandas UDF

In [87]:
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    return series + 1

df.select(pandas_plus_one(df.a)).show()

+------------------+
|pandas_plus_one(a)|
+------------------+
|                 2|
|                 3|
|                 5|
+------------------+



Using the DataFrame below, calculate the average of `v1` and `v2` for each color.

```python
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
```

In [130]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])

df. groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [90]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Title: string (nullable = true)



Using the previous DF and a regular Pandas function, group by color and update `v1` by adding the group's mean to each value.

In [14]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 + pandas_df.v1.mean())

df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot| 12| 60|
| blue|banana|  5| 20|
| blue| grape|  7| 40|
|  red|banana|  5| 10|
|  red|carrot|  7| 30|
|  red|carrot|  9| 50|
|  red|banana| 11| 70|
|  red| grape| 12| 80|
+-----+------+---+---+



Left join `df1` and `df2` by key distance.

In [104]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0), (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ('time', 'id', 'v1'))

df2 = spark.createDataFrame(
    [(20000101, 1, 'x'), (20000101, 2, 'y')],
    ('time', 'id', 'v2'))

df1.show()
df2.show()

+--------+---+---+
|    time| id| v1|
+--------+---+---+
|20000101|  1|1.0|
|20000101|  2|2.0|
|20000102|  1|3.0|
|20000102|  2|4.0|
+--------+---+---+

+--------+---+---+
|    time| id| v2|
+--------+---+---+
|20000101|  1|  x|
|20000101|  2|  y|
+--------+---+---+



In [34]:
def asof_join(l, r):
    return pd.merge_asof(l, r, on='time', by='id')

df1.groupby('id').cogroup(df2.groupby('id'))\
                 .applyInPandas(asof_join, schema='time int, id int, v1 double, v2 string')\
                 .show()

+--------+---+---+---+
|    time| id| v1| v2|
+--------+---+---+---+
|20000101|  1|1.0|  x|
|20000102|  1|3.0|  x|
|20000101|  2|2.0|  y|
|20000102|  2|4.0|  y|
+--------+---+---+---+



### SQL statements

In [107]:
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



Count rows in table

In [133]:
df.createOrReplaceTempView("TableA")

In [128]:
spark.sql("SELECT count(*) from TableA").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



Add one to `v1` by using a Pandas UDF

In [134]:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(v1) FROM TableA").show()

22/06/20 11:25:45 WARN SimpleFunctionRegistry: The function add_one replaced a previously registered function.


+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+



In [135]:
df.selectExpr("v1", "add_one(v1) v1_add1").show()

+---+-------+
| v1|v1_add1|
+---+-------+
|  1|      2|
|  2|      3|
|  3|      4|
|  4|      5|
|  5|      6|
|  6|      7|
|  7|      8|
|  8|      9|
+---+-------+



Alterntatively, we can use SQL expressions as PySpark columns, usindg `expr`:
- Add one to column `v1`.
- Test if table has more than 0 rows

In [136]:
from pyspark.sql.functions import expr
df.select(expr('count(*) > 0 positive_rows')).show()

+-------------+
|positive_rows|
+-------------+
|         true|
+-------------+

